# Groupby, Pivot, Merge

In [1]:
import pandas as pd 
import numpy as np 

univ = ['AAPL','MSFT','BAC','GS']
dates = pd.date_range('20110101','20201231')
df = pd.DataFrame(np.random.randn(len(dates),len(univ)),index=dates,columns=univ)
df

,AAPL,MSFT,BAC,GS
2011-01-01,-0.661320,-0.483994,1.456499,-1.058578
2011-01-02,-0.248735,0.762345,-0.205302,-0.573180
2011-01-03,-0.553902,1.217359,0.996110,1.332993
2011-01-04,-2.957304,-1.268012,1.784480,1.272902
2011-01-05,0.180272,0.546704,1.526237,-1.588706
...,...,...,...,...
2020-12-27,-0.832337,-0.157373,0.303128,0.871913
2020-12-28,-0.500830,-0.118648,0.331402,1.420394
2020-12-29,0.951564,-0.103776,0.255349,-0.518534
2020-12-30,0.140910,0.071293,-0.771591,0.812735


In [2]:
sector = {'AAPL':'Tech','MSFT':'Tech','BAC':'Fin','GS':'Fin'}
data=[]
for dt in dates:
    for x in univ:
        data.append([x,sector[x],dt,np.random.randn(),np.random.randn()])
data = pd.DataFrame(data,columns=['ticker','sector','date','signal1','signal2'])
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.805907,-0.226885
1,MSFT,Tech,2011-01-01,0.070573,0.801604
2,BAC,Fin,2011-01-01,0.204576,-0.614069
3,GS,Fin,2011-01-01,0.459852,1.528903
4,AAPL,Tech,2011-01-02,-0.407962,-0.848494
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,-3.006238,3.046418
14608,AAPL,Tech,2020-12-31,0.864428,1.334954
14609,MSFT,Tech,2020-12-31,-0.820475,1.333956
14610,BAC,Fin,2020-12-31,-1.486485,-0.896486


## Groupby

In [6]:
# whats the average signal of each ticker?
data.groupby('ticker')[['signal1', 'signal2']].mean()

,signal1,signal2
ticker,,
AAPL,-0.005673,0.004120
BAC,0.000793,-0.005917
GS,0.005786,-0.028829
MSFT,-0.007307,-0.015311


In [11]:
# whats the average signal of each sector on each day?
klk = data.groupby(['sector','date'])[['signal1', 'signal2']].mean()
klk.index

MultiIndex([( 'Fin', '2011-01-01'),
            ( 'Fin', '2011-01-02'),
            ( 'Fin', '2011-01-03'),
            ( 'Fin', '2011-01-04'),
            ( 'Fin', '2011-01-05'),
            ( 'Fin', '2011-01-06'),
            ( 'Fin', '2011-01-07'),
            ( 'Fin', '2011-01-08'),
            ( 'Fin', '2011-01-09'),
            ( 'Fin', '2011-01-10'),
            ...
            ('Tech', '2020-12-22'),
            ('Tech', '2020-12-23'),
            ('Tech', '2020-12-24'),
            ('Tech', '2020-12-25'),
            ('Tech', '2020-12-26'),
            ('Tech', '2020-12-27'),
            ('Tech', '2020-12-28'),
            ('Tech', '2020-12-29'),
            ('Tech', '2020-12-30'),
            ('Tech', '2020-12-31')],
           names=['sector', 'date'], length=7306)

In [12]:
# can index before applying the function
data.groupby(['sector','date'])[['signal1']].mean()

signal1
sector date                
Fin    2011-01-01  0.332214
       2011-01-02  1.896882
       2011-01-03 -0.840119
       2011-01-04  0.406362
       2011-01-05  0.433787
...                     ...
Tech   2020-12-27 -0.554642
       2020-12-28 -0.050215
       2020-12-29  1.066636
       2020-12-30  1.248044
       2020-12-31  0.021977

[7306 rows x 1 columns]

In [14]:
# use apply with groupby to pass an arbitrary function
def max_minus_min(x):
    return x.max()-x.min()

data.groupby(['sector','date'])[['signal1','signal2']].apply(max_minus_min)

signal1   signal2
sector date                          
Fin    2011-01-01  0.255275  2.142972
       2011-01-02  0.147126  0.721942
       2011-01-03  0.716304  0.985195
       2011-01-04  2.891956  2.574893
       2011-01-05  2.008533  0.943243
...                     ...       ...
Tech   2020-12-27  0.440544  0.486691
       2020-12-28  0.798344  0.472164
       2020-12-29  0.316642  0.200882
       2020-12-30  0.411006  0.259150
       2020-12-31  1.684903  0.000997

[7306 rows x 2 columns]

In [15]:
# return a dataframe instead of a series 
def demean(x):
    return x - x.mean()

data.groupby(['sector','date'])[['signal1','signal2']].apply(demean)

signal1   signal2
sector date                                
Fin    2011-01-01 2     -0.127638 -1.071486
                  3      0.127638  1.071486
       2011-01-02 6     -0.073563  0.360971
                  7      0.073563 -0.360971
       2011-01-03 10     0.358152 -0.492598
...                           ...       ...
Tech   2020-12-29 14601  0.158321 -0.100441
       2020-12-30 14604  0.205503 -0.129575
                  14605 -0.205503  0.129575
       2020-12-31 14608  0.842451  0.000499
                  14609 -0.842451 -0.000499

[14612 rows x 2 columns]

In [16]:
df = data.groupby(['sector','date'])[['signal1','signal2']].mean()
df

signal1   signal2
sector date                          
Fin    2011-01-01  0.332214  0.457417
       2011-01-02  1.896882  0.101598
       2011-01-03 -0.840119 -0.543106
       2011-01-04  0.406362  0.652004
       2011-01-05  0.433787 -0.967118
...                     ...       ...
Tech   2020-12-27 -0.554642 -0.116514
       2020-12-28 -0.050215  0.529590
       2020-12-29  1.066636 -1.423242
       2020-12-30  1.248044 -1.100371
       2020-12-31  0.021977  1.334455

[7306 rows x 2 columns]

In [17]:
df.groupby(level=0).mean()

,signal1,signal2
sector,,
Fin,0.00329,-0.017373
Tech,-0.00649,-0.005595


In [21]:
month = [x.month for x in data['date']]

In [22]:
data.groupby(month)[['signal1, signal2']].mean()

KeyError: "Columns not found: 'signal1, signal2'"

In [23]:
for key,val in data.groupby('sector'):
    print (key)

Fin
Tech


In [24]:
# iterating in a groupby
for key,val in data.groupby('sector'):
    print (val)

      ticker sector       date   signal1   signal2
2        BAC    Fin 2011-01-01  0.204576 -0.614069
3         GS    Fin 2011-01-01  0.459852  1.528903
6        BAC    Fin 2011-01-02  1.823319  0.462569
7         GS    Fin 2011-01-02  1.970445 -0.259373
10       BAC    Fin 2011-01-03 -0.481967 -1.035704
...      ...    ...        ...       ...       ...
14603     GS    Fin 2020-12-29  0.605835  0.439401
14606    BAC    Fin 2020-12-30 -1.639666 -1.768419
14607     GS    Fin 2020-12-30 -3.006238  3.046418
14610    BAC    Fin 2020-12-31 -1.486485 -0.896486
14611     GS    Fin 2020-12-31  0.335486 -0.332652

[7306 rows x 5 columns]
      ticker sector       date   signal1   signal2
0       AAPL   Tech 2011-01-01 -0.805907 -0.226885
1       MSFT   Tech 2011-01-01  0.070573  0.801604
4       AAPL   Tech 2011-01-02 -0.407962 -0.848494
5       MSFT   Tech 2011-01-02  1.178689  1.230866
8       AAPL   Tech 2011-01-03  1.367206 -0.410473
...      ...    ...        ...       ...       ...
14601 

## Pivot

In [25]:
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.805907,-0.226885
1,MSFT,Tech,2011-01-01,0.070573,0.801604
2,BAC,Fin,2011-01-01,0.204576,-0.614069
3,GS,Fin,2011-01-01,0.459852,1.528903
4,AAPL,Tech,2011-01-02,-0.407962,-0.848494
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,-3.006238,3.046418
14608,AAPL,Tech,2020-12-31,0.864428,1.334954
14609,MSFT,Tech,2020-12-31,-0.820475,1.333956
14610,BAC,Fin,2020-12-31,-1.486485,-0.896486


In [26]:
# put Signal1 into a df with columns tickers and rows dates
df1 = data.set_index(['date','ticker'])['signal1'].unstack(level=1)
df1

ticker,AAPL,BAC,GS,MSFT
date,,,,
2011-01-01,-0.805907,0.204576,0.459852,0.070573
2011-01-02,-0.407962,1.823319,1.970445,1.178689
2011-01-03,1.367206,-0.481967,-1.198271,-0.224781
2011-01-04,1.518218,-1.039616,1.852340,1.121633
2011-01-05,0.129083,1.438054,-0.570479,0.271483
...,...,...,...,...
2020-12-27,-0.334370,-0.343151,1.398287,-0.774914
2020-12-28,0.348956,1.044393,-1.130114,-0.449387
2020-12-29,0.908314,0.292735,0.605835,1.224957


In [27]:
df2 = data.pivot_table(index='date',columns='ticker',values='signal1')
df2

ticker,AAPL,BAC,GS,MSFT
date,,,,
2011-01-01,-0.805907,0.204576,0.459852,0.070573
2011-01-02,-0.407962,1.823319,1.970445,1.178689
2011-01-03,1.367206,-0.481967,-1.198271,-0.224781
2011-01-04,1.518218,-1.039616,1.852340,1.121633
2011-01-05,0.129083,1.438054,-0.570479,0.271483
...,...,...,...,...
2020-12-27,-0.334370,-0.343151,1.398287,-0.774914
2020-12-28,0.348956,1.044393,-1.130114,-0.449387
2020-12-29,0.908314,0.292735,0.605835,1.224957


In [28]:
df1.equals(df2)

True

In [29]:
data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.mean)

sector,Fin,Tech
date,,
2011-01-01,0.332214,-0.367667
2011-01-02,1.896882,0.385364
2011-01-03,-0.840119,0.571212
2011-01-04,0.406362,1.319926
2011-01-05,0.433787,0.200283
...,...,...
2020-12-27,0.527568,-0.554642
2020-12-28,-0.042861,-0.050215
2020-12-29,0.449285,1.066636


In [30]:
data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.max)

sector,Fin,Tech
date,,
2011-01-01,0.459852,0.070573
2011-01-02,1.970445,1.178689
2011-01-03,-0.481967,1.367206
2011-01-04,1.852340,1.518218
2011-01-05,1.438054,0.271483
...,...,...
2020-12-27,1.398287,-0.334370
2020-12-28,1.044393,0.348956
2020-12-29,0.605835,1.224957


## Merge
Check out documentation to learn more: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [31]:
data2=[]
for dt in dates:
    for x in ['Tech','Fin']:
        data2.append([x,dt,np.random.randn(),np.random.randn()])
        
data2 = pd.DataFrame(data2,columns=['sector','date','signal3','signal4'])
data2

,sector,date,signal3,signal4
0,Tech,2011-01-01,-1.653200,0.468908
1,Fin,2011-01-01,-1.254231,0.162715
2,Tech,2011-01-02,0.577937,-0.682863
3,Fin,2011-01-02,-3.520048,-0.430568
4,Tech,2011-01-03,-0.808520,-0.386761
...,...,...,...,...
7301,Fin,2020-12-29,0.714526,-1.046780
7302,Tech,2020-12-30,0.075920,-2.542229
7303,Fin,2020-12-30,1.131685,-0.204354
7304,Tech,2020-12-31,0.292801,-0.990760


In [32]:
data.merge(data2,left_on=['sector','date'],right_on=['sector','date'])

,ticker,sector,date,signal1,signal2,signal3,signal4
0,AAPL,Tech,2011-01-01,-0.805907,-0.226885,-1.653200,0.468908
1,MSFT,Tech,2011-01-01,0.070573,0.801604,-1.653200,0.468908
2,BAC,Fin,2011-01-01,0.204576,-0.614069,-1.254231,0.162715
3,GS,Fin,2011-01-01,0.459852,1.528903,-1.254231,0.162715
4,AAPL,Tech,2011-01-02,-0.407962,-0.848494,0.577937,-0.682863
...,...,...,...,...,...,...,...
14607,GS,Fin,2020-12-30,-3.006238,3.046418,1.131685,-0.204354
14608,AAPL,Tech,2020-12-31,0.864428,1.334954,0.292801,-0.990760
14609,MSFT,Tech,2020-12-31,-0.820475,1.333956,0.292801,-0.990760
14610,BAC,Fin,2020-12-31,-1.486485,-0.896486,0.018363,-0.007361
